In [ ]:

# Generación a del algoritmo de PRNG básico (el de generador de numeros pseudoaleatorios)

import random

def generar_keystream(seed: str, longitud: int) -> bytes:
    # Passing the current time as the seed value

    random.seed(seed)

    arreglodebytes = bytearray()
    for _ in range(longitud):
        numero = random.randint(0, 255)  # full byte range
        arreglodebytes.append(numero)


    return bytes(arreglodebytes)
    


valor_a_investigar = "HyperPlay"
seed = "Estoesunasemilla"

print(generar_keystream( seed ,len(valor_a_investigar)))


b'\xc7\t\xe4\xado\xc4\x05\x97\x91'
b'\xc7\t\xe4\xado\xc4\x05\x97\x91'
